In [3]:
!pip install delu
!pip install ucimlrepo
!pip install gdown
!pip install rtdl_num_embeddings
!pip install wandb
# !pip install optuna
!git clone https://github.com/gbulgakov/testing-kan.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [4]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import optuna
import numpy as np
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
from typing import Literal, Optional
from torch import Tensor
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import rtdl_num_embeddings
import delu
from IPython.display import FileLink
from tqdm import tqdm
import wandb
wandb.login(key='936d887040f82c8da3d87f5207c4178259c7b922')


# наши импорты
sys.path.append('/kaggle/working/testing-kan/optimizers')
sys.path.append('/kaggle/working/testing-kan')
from ademamix import AdEMAMix
from mars import MARS
from muon import Muon
from efficient_kan import KAN
from utils import utils

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gbulgakov (georgy-bulgakov) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
# !gdown 1xvRa_-OEeG6xNRYE5V5iAfTwyWM1NiLl # otto
# !gdown 1tYyhbHdYs_8I9jvXznMoeUAfBzwitaax # house
# !gdown 1hy1dOAL2SE-XZSuMcjLcVgml2CoYkF9q # higgs-small
# !gdown 1hr076cK9QFxH6YZRg5V4av-H7IAve59r # gesture
# !gdown 1ZNScy5fgqtgudT6MZ4EjLt1nwdqirtmX # fb-comments
!gdown 1s0w7gnhiwBCkF49Wdi_cUDpUtXlz2_6q # eye
# !gdown 1T04iP04UGVo95Om84ww1Ed8AFNziOaeY # covtype
!gdown 1GOkNlinj4zHVSNKbqjN1rR4cvsAf2IgR # churn
# !gdown 11B-l4EasJkclK_Q-RBcxYfGJLSvz-v5c # california
# !gdown 1p8uqDPMfRlFIc69m7iikS6wGkA6JGj1H # adult

Downloading...
From: https://drive.google.com/uc?id=1s0w7gnhiwBCkF49Wdi_cUDpUtXlz2_6q
To: /kaggle/working/eye.zip
100%|████████████████████████████████████████| 534k/534k [00:00<00:00, 1.62MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GOkNlinj4zHVSNKbqjN1rR4cvsAf2IgR
To: /kaggle/working/churn.zip
100%|████████████████████████████████████████| 453k/453k [00:00<00:00, 1.46MB/s]


Я вынес в файл ``utils`` функции ``count_parameters``,``load_dataset``, ``seed_everything``,  ``write_results``. 
Модели не меняем.

In [6]:
DATASETS = ['adult', 'california', 'churn', 'covtype', 'eye', 'fb-comments',
           'gesture', 'higgs-small', 'house', 'microsoft', 'otto', 'santander']


BATCH_SIZES = {'gesture' : 128, 'churn' : 128, 'california' : 256, 'house' : 256, 'adult' : 256, 'otto' : 512, 
               'higgs-small' : 512, 'fb-comments' : 512, 'santander' : 1024, 'covtype' : 1024, 'microsoft' : 1024, 'eye': 128}

REGRESSION = ['house', 'fb-comments', 'microsoft', 'california']
MULTICLASS = ['covtype', 'eye', 'gesture', 'otto']
BINCLASS = ['adult', 'churn', 'higgs-small', 'santander']

OPTIMIZERS = {'adamw' : torch.optim.AdamW,
              'mars' : MARS,
              'ademamix' : AdEMAMix,
              'muon' : Muon}

Модели не меняем.

In [7]:
class MLP(nn.Sequential):
    def __init__(self, layers, dropout):
        super(MLP, self).__init__()
        
        total_layers = []
        for n_in, n_out in zip(layers[:-2], layers[1:-1]):
            total_layers.append(nn.Linear(n_in, n_out))
            total_layers.append(nn.SiLU(inplace=False))
            total_layers.append(nn.Dropout(dropout, inplace=False))
        total_layers.append(nn.Linear(layers[-2], layers[-1])) # выходной слой

        self.classifier = nn.Sequential(*total_layers)


In [8]:
class ModelWithEmbedding(nn.Module):
    def __init__(
        self,
        n_cont_features,
        d_embedding,
        emb_name,
        backbone_model,
        bins, sigma=None # словарь всех необязательных параметров, например sigma, bins
    ) -> None:
        super().__init__()
        self.d_embedding = d_embedding
        self.emb_name = emb_name
        
        if emb_name == 'periodic':
            self.cont_embeddings = rtdl_num_embeddings.PeriodicEmbeddings(
                n_cont_features, d_embedding, frequency_init_scale=sigma, lite=True
            )
            
        if emb_name == 'piecewiselinearq' or emb_name == 'piecewiselineart' or emb_name == 'PLE-Q' or emb_name == 'PLE-T':
            self.cont_embeddings = rtdl_num_embeddings.PiecewiseLinearEmbeddings(
                d_embedding=d_embedding, activation=False, version='B', bins=bins
            )

        self.backbone = backbone_model
    
    def forward(
        self,
        x_num : Tensor,
        x_cat : Optional[Tensor] = None
    ) -> Tensor:
        x = []
        # Step 1. Embed the continuous features.
        # Flattening is needed for MLP-like models.
        if self.emb_name != 'none':
              x_num = self.cont_embeddings(x_num)
        x.append(x_num.flatten(1))
        
        #categorical features do not need embeddings
        if x_cat is not None:
            x.append(x_cat.flatten(1))
        
        x = torch.column_stack(x)
        return self.backbone(x)

Заготовим функцию, которая создает конфигурацию тюнинга.

Подбор параметров не меняем, тюним архитектуру сеток, ``lr``, ``weight_decay``.

In [9]:
def get_sweep_config(model_name, emb_name, task_type, sweep_name=''):
    metric = {}
    if task_type == 'regression':
        metric = {'name' : 'val_loss', 'goal' : 'minimize'}
    else:
        metric = {'name' : 'val_acc', 'goal' : 'maximize'}
    
    max_log_width = (7 if model_name == 'kan' else 11)
    params = {
        'lr' : {
            'distribution' : 'log_uniform_values',
            'min' : 1e-5,
            'max' : 1e-2
        },
        'weight_decay' : {
            'distribution' : 'log_uniform_values',
            'min' : 1e-6,
            'max' : 1e-3
        },
        'hidden_layers' : { # скрытые слои
            'values' : [1, 2, 3, 4]
        },
        'layer_width' : {
            'values' : [2 ** i for i in range(max_log_width)]
        },
        'grid_size' : {
            'values' : [i for i in range(3, 30, 2)]
        }
    }
    if model_name == 'mlp':
        params.update({
            'use_dropout' : {'values' : [True, False]},
            'dropout' : {'values' : [i / 100 for i in range(0, 55, 5)]}
        })
    if emb_name != 'none':
        params.update({
            'd_embedding' : {'values' : [2 ** i for i in range(1, 8)]}
        })
    if emb_name == 'periodic':
        params.update({
            'sigma' : {
                'distribution' : 'uniform',
                'min' : 0.01,
                'max' : 100
            }
        })
    
    config = {
        'method' : 'random',
        'metric' : metric,
        'parameters' : params,
        'name' : sweep_name
    }
    return config

In [10]:
def get_test_config(model_name, emb_name, task_type, sweep_name=''):
    metric = {} # чисто технический параметр
    if task_type == 'regression':
        metric = {'name' : 'val_loss', 'goal' : 'minimize'}
    else:
        metric = {'name' : 'val_acc', 'goal' : 'maximize'}
    params = {
        'seed' : {
            'values' : [i for i in range(10)]
        }
    }
    config = {
        'method' : 'grid',
        'metric' : metric,
        'parameters' : params,
        'name' : sweep_name
    }
    return config

В ``train`` теперь передаем целиком ``optimizer``.  Кроме того, разнесем обучение по эпохам.

In [11]:
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.optim import AdamW, Adam
from torch.optim.lr_scheduler import ExponentialLR, StepLR, CosineAnnealingLR
from torch.nn import MSELoss
import torch.nn as nn
import time


def apply_model(batch: dict[str, Tensor], model) -> Tensor:
    return model(batch['X_num'], batch.get('X_cat')).squeeze(-1)

def train_epoch(model, device, dataset, loss_fn, optimizer, scheduler):

    dataset_name = dataset['info']['id'].split('--')[0]
    task_type = dataset['info']['task_type']
    batch_size = BATCH_SIZES[dataset_name]

    model.to(device)
    model.train()
    train_loss = 0.0
    pred = []
    gt = [] # настоящие таргеты
    start_time = time.time()

    for data in delu.iter_batches(dataset['train'], shuffle=True, batch_size=batch_size):
        for key, tensor in data.items():
            data[key] = tensor.to(device)
        # обучение
        optimizer.zero_grad()
        output = apply_model(data, model)
        if task_type == 'multiclass':
            data['y'] = data['y'].long()
        loss_value = loss_fn(output, data['y']) 
        loss_value.backward()
        optimizer.step()
        # сохранение истории
        train_loss += loss_value.item()
        if output.dim() > 1:
            pred.append(output.argmax(1))
        else:
            pred.append(output >= 0.5)
        gt.append(data['y'])
    scheduler.step()

    end_time = time.time()
    epoch_time = end_time - start_time

    num_batches = dataset['train']['y'].shape[0] // batch_size + 1
    pred = torch.cat(pred)
    gt = torch.cat(gt)
    train_accuracy = (pred == gt).float().mean().item()

    return train_loss / num_batches, train_accuracy, epoch_time # с нормировкой
    
def validate(model, device, dataset, loss_fn, part='val'):
    model.eval()
    model.to(device)
    val_loss = 0.0

    pred = []
    gt = [] # настоящие таргеты

    dataset_name = dataset['info']['id'].split('--')[0]
    task_type = dataset['info']['task_type']
    batch_size = BATCH_SIZES[dataset_name]

    with torch.no_grad():
        start_time = time.time()
        for data in delu.iter_batches(dataset[part], shuffle=False, batch_size=batch_size):
            for key, tensor in data.items():
                data[key] = tensor.to(device)
            output = apply_model(data, model)
            if task_type == 'multiclass':
                data['y'] = data['y'].long()
            val_loss += loss_fn(output, data['y']).item()
            if output.dim() > 1:
                pred.append(output.argmax(1))
            else:
                pred.append(output >= 0.5)
            gt.append(data['y'])
        end_time = time.time()
        val_time = end_time - start_time
        

    num_batches = dataset[part]['y'].shape[0] // batch_size + 1
    pred = torch.cat(pred)
    gt = torch.cat(gt)
    val_accuracy = (pred == gt).float().mean().item()

    return val_loss / num_batches, val_accuracy, val_time # с нормировкой

Возможно ``model.to`` лучше вызывать 1 раз.

In [12]:
def train(
    epochs, model, model_emb_name,
    device, dataset, loss_fn,
    optimizer,
    optimizer_name=None,
):
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    model.to(device)
    dataset_name = dataset['info']['id'].split('--')[0]
    task_type = dataset['info']['task_type']
    batch_size = BATCH_SIZES[dataset_name]

    train_times = []
    val_times = []
    for epoch in tqdm(range(epochs), desc = f'{model_emb_name}_{optimizer_name} on {dataset_name}'):
        train_loss, train_acc, train_time = train_epoch(model, device, dataset, loss_fn, optimizer, scheduler)
        val_loss, val_acc, val_time = validate(model, device, dataset, loss_fn)

        wandb.log({
            'epoch' : epoch,
            'train_loss' : train_loss,
            'train_acc' : train_acc,
            'val_loss' : val_loss,
            'val_acc' : val_acc,
            'lr' : scheduler.get_last_lr()[0]
        })
        train_times.append(train_time)
        val_times.append(val_time)

    # размерность входа backbone
    in_features = dataset['train']['X_num'].shape[1]  # Количество числовых признаков
    if 'X_cat' in dataset['train']:
        in_features += dataset['train']['X_cat'].shape[1]  # Добавляем категориальные признаки

    
    wandb.log({
        'train_epoch_time' : sum(train_times) / epochs,
        'val_epoch_time' : sum(val_times) / epochs,
        'num_params' : utils.count_parameters(model),
        'in_features' : in_features,
        'out_features' : (1 if task_type != 'multiclass' else dataset['info']['n_classes'])
        # ширины и так будут залоггированы
    })

Подготовку модели нужно переписать на языке ``wandb.config``. Все параметры, кроме ``hidden_layers`` вместо ``n_layers``, остаются такими же.

In [13]:
def model_init_preparation(config, dataset, model_name, emb_name):
    dataset_info = dataset['info']
    num_cont_cols = dataset['train']['X_num'].shape[1]
    num_cat_cols = 0
    if dataset_info['n_cat_features'] > 0:
        num_cat_cols = dataset['train']['X_cat'].shape[1]
    num_classes = 1
    if dataset_info['task_type'] == 'multiclass':
        num_classes = dataset_info['n_classes']
        

    # создание модели
    layer_widths = list(range(config['hidden_layers'] + 2))
    
    if emb_name != 'none':
        layer_widths[0] = num_cont_cols * config['d_embedding'] + num_cat_cols
    else:
        layer_widths[0] = num_cont_cols + num_cat_cols
    layer_widths[1:-1] = [config['layer_width'] for i in range(config['hidden_layers'])] #скрытые слои
    layer_widths[-1] = num_classes
            
    if model_name == 'kan':
        backbone = KAN(layer_widths, grid_size=config['grid_size'], batch_norm=True)
    elif model_name == 'mlp':
        dropout = (config['dropout'] if config['use_dropout'] else 0)
        backbone = MLP(layer_widths, dropout)
    
    # создание эмбеддингов
    if emb_name == 'piecewiselinearq' or emb_name == 'PLE-Q':
        bins = rtdl_num_embeddings.compute_bins(dataset['train']['X_num'], n_bins=config['d_embedding'])
    elif emb_name == 'piecewiselineart' or emb_name == 'PLE-T': # это мы  больше не используем
        tree_kwargs = {'min_samples_leaf': 64, 'min_impurity_decrease': 1e-4} #возможно стоит тюнить
        bins = rtdl_num_embeddings.compute_bins(X=dataset['train']['X_num'], y=dataset['train']['y'], n_bins=config['d_embedding'], regression=True, tree_kwargs=tree_kwargs)
    else:
        bins = None
            
    task_type = dataset_info['task_type']
    loss_fn = None
    
    if task_type == 'binclass':
        loss_fn = F.binary_cross_entropy_with_logits
    elif task_type == 'multiclass':
        loss_fn = F.cross_entropy
    else:
        loss_fn =  F.mse_loss
        
    return layer_widths, backbone, bins, loss_fn
    


Функцию, запускающую модель, дополним созданием определеннного ``optimizer``. Это тоже переписали на язык ``wandb.config``.

In [14]:
def get_optimizer(optim_name, model_params, config):
    optim_class = OPTIMIZERS[optim_name]
    optim_kwargs = {'lr' : config['lr']}
    if optim_name != 'muon': # это на будущее
        optim_kwargs['weight_decay'] = config['weight_decay']
    return optim_class(model_params, **optim_kwargs)



Функция для тюнинга!

In [15]:
def wandb_tuning(project_name, dataset_name, 
                 model_name, emb_name, optim_name, 
                 dataset, num_epochs=10, num_trials=30):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dataset_info = dataset['info']
    num_cont_cols = dataset['train']['X_num'].shape[1]
    sweep_name = f'tuning {model_name}_{emb_name}_{optim_name} on {dataset_name}'
    sweep_config = get_sweep_config(model_name, emb_name, dataset_info['task_type'], 
                                    f'tuning {model_name}_{emb_name}_{optim_name} on {dataset_name}')
    
    # просто оборачиваем нашу train
    def sweep_wrapper():
        with wandb.init(
            project=f'{project_name}',
            group=f'dataset_{dataset_name}',
            tags=[f'model_{model_name}', f'emb_{emb_name}', f'optim_{optim_name}', f'dataset_{dataset_name}', 'tuning'],
            config=sweep_config
        ) as run:
            config = wandb.config
            _, backbone, bins, loss_fn = model_init_preparation(
                config=config,
                dataset=dataset,
                model_name=model_name,
                emb_name=emb_name
            )
            model = ModelWithEmbedding(
                n_cont_features=num_cont_cols,
                d_embedding=config.get('d_embedding', None),
                emb_name=emb_name,
                backbone_model=backbone,
                bins=bins, 
                sigma=config.get('sigma', None)
            )
            train(
                epochs=num_epochs,
                model=model,
                model_emb_name=f'{model_name}_{emb_name}_{optim_name}',
                device=device,
                dataset=dataset,
                loss_fn=loss_fn,
                optimizer=get_optimizer(optim_name, model.parameters(), config),
                optimizer_name = optim_name
            )
    sweep_config = get_sweep_config(model_name, emb_name, task_type=dataset_info['task_type'])
    sweep_id = wandb.sweep(sweep=sweep_config,
                           project=f'{project_name}',
                           entity='georgy-bulgakov') 
    wandb.agent(sweep_id, sweep_wrapper, count=num_trials)
    return sweep_id

Функция для тестирования на разных сидах.

In [20]:
def test_best_model(best_params, project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs=10):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_cont_cols = dataset['train']['X_num'].shape[1]
    d_embedding = best_params.get('d_embedding', None)
    sigma = best_params.get('sigma', None)

    # подготовка логирования
    test_accuracies = []
    test_losses = []
    test_times = []
    train_times = []

    testing_config = get_test_config(model_name, emb_name, dataset['info']['task_type'], 
                                    f'testing {model_name}_{emb_name}_{optim_name} on {dataset_name}')
    # обертка тестирования
    def test_wrapper():
        with wandb.init(
            project=f'{project_name}',
            group=f'dataset_{dataset_name}',
            tags=[f'model_{model_name}', f'emb_{emb_name}', f'optim_{optim_name}', f'dataset_{dataset_name}', 'testing'],
            config=testing_config
        ) as run:
            config = wandb.config
            # seed + подготовка модели
            utils.seed_everything(config['seed'])
            layers, backbone, bins, loss_fn = model_init_preparation(
                config=best_params,
                dataset=dataset,
                model_name=model_name,
                emb_name=emb_name
            )
            model = ModelWithEmbedding(
                n_cont_features=num_cont_cols,
                d_embedding=d_embedding,
                emb_name=emb_name,
                backbone_model=backbone,
                bins=bins, 
                sigma=sigma
            )

            start_time = time.time()
            train(
                epochs=num_epochs,
                model=model,
                model_emb_name=f'{model_name}_{emb_name}_{optim_name}',
                device=device,
                dataset=dataset,
                loss_fn=loss_fn,
                optimizer=get_optimizer(optim_name, model.parameters(), best_params)
            )
            end_time = time.time()
            test_loss, test_acc, test_time = validate(model, device, dataset, loss_fn, 'test')
            # Логируем результаты для каждого сида
            run.log({
                'test_loss': test_loss,
                'test_acc': test_acc,
                'full_train_time': end_time - start_time,
                'test_time': test_time,
                'seed': config['seed']
            })

            test_accuracies.append(test_acc)
            test_losses.append(test_loss)
            test_times.append(test_time)
            train_times.append(end_time - start_time)

    test_id = wandb.sweep(sweep=testing_config,
                           project=f'{project_name}',
                           entity='georgy-bulgakov') 
    wandb.agent(test_id, test_wrapper)

    # Создаем финальный run для агрегированных результатов
    with wandb.init(
        project=f"{project_name}",
        group=f'dataset_{dataset_name}',
        name="aggregated_results",
        tags=[f'model_{model_name}', f'emb_{emb_name}', f'optim_{optim_name}', f'dataset_{dataset_name}', 'testing'],
        config=best_params
    ) as run:
        stats = {
            'model' : f'{model_name}_{emb_name}_{optim_name}',
            'mean_test_acc': np.mean(test_accuracies),
            'std_test_acc': np.std(test_accuracies),
            'mean_test_loss': np.mean(test_losses),
            'std_test_loss': np.std(test_losses),
            'mean_test_time': np.mean(test_times),
            'mean_train_time': np.mean(train_times),
            'all_test_accs': test_accuracies,
            'all_test_losses': test_losses,
            'all_test_times': test_times,
            'all_train_times': train_times
        }
        
        run.log(stats)
        stats['name'] = f'{model_name}_{emb_name}_{optim_name}'
        
    keys = ['model', 'mean_test_acc', 'std_test_acc', 'mean_test_loss', 'std_test_loss', 'mean_test_time', 'mean_train_time']
    return {key : stats[key] for key in keys} # чисто технически для удобства

epoch,▁▃▆█
in_features,▁
lr,█▅▂▁
num_params,▁
out_features,▁
train_acc,▁▁▁▁
train_epoch_time,▁
train_loss,█▄▂▁
val_acc,▁▁▁▁
val_epoch_time,▁
val_loss,█▂▂▁


In [21]:
from IPython.display import clear_output
def run_single_model(project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs):
    sweep_id = wandb_tuning(project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs)
    clear_output(wait=True)
    # вспоминаем лучшие параметры
    api = wandb.Api()
    sweep = api.sweep(f'georgy-bulgakov/{project_name}/{sweep_id}')
    runs = sweep.runs
    best_run = None
    if dataset['info']['task_type'] == 'regression':
        best_run = min(runs, key=lambda run : run.summary.get('val_loss'))
    else:
        best_run = max(runs, key=lambda run : run.summary.get('val_acc'))
    best_params = best_run.config
    
    stats = test_best_model(best_params, project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs)
    return stats, best_params

In [22]:
import pandas as pd
import pickle
def run_single_dataset(project_name, dataset_name, optim_names, emb_names, model_names, num_epochs):
    # dataset_type = dataset_info['type']
    dataset = utils.load_dataset(dataset_name)
    results = []
    pkl_logs = {}

    for model_name in model_names: # можно оставить только kan, тогда model_names = ['kan']
        for optim_name in optim_names:
            for emb_name in emb_names:
                stats, best_params = run_single_model(project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs)
                results.append(stats)
                clear_output(wait=True)
                pkl_logs[f'{model_name}_{emb_name}_{optim_name}'] = (stats | best_params)

    
    with wandb.init(
        project=project_name,
        group=f'dataset_{dataset_name}',
        name='models_comparison'
    ) as run:
        run.log({
            'Final Table' : wandb.Table(dataframe=pd.DataFrame(results))
        })

    with open(f'{dataset_name}.pkl', 'wb') as f:
        pickle.dump(pkl_logs, f)
    return results



In [23]:
optim_names = ['adamw']
model_names = ['kan']
emb_names = ['none', 'periodic']
project_name = 'TAB-KAN study'

for dataset_name in ['churn', 'eye']:
    run_single_dataset(project_name, dataset_name, optim_names, emb_names, model_names, 10)